# Import Library

In [ ]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from scipy.sparse import hstack # Diimpor tapi tidak digunakan dalam kode ini
from sklearn.feature_extraction.text import TfidfVectorizer # Diimpor tapi tidak digunakan dalam kode ini
from indo_normalizer import Normalizer
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# --- Download Data NLTK ---
# Mengunduh resource NLTK yang diperlukan
nltk.download('punkt')
nltk.download('punkt_tab') # Resource ini mungkin tidak standar atau diperlukan
nltk.download('stopwords')

# LOAD DATASET

In [ ]:
FILE_PATH = "tweet.csv"

# Load Dataset dan pastikan semua kolom bertipe string
df = pd.read_csv(FILE_PATH).astype(str)

# Cleaning Text

In [ ]:
# --- Text Normalizer ---
normalizer = Normalizer()

# Daftar stopwords standar dari NLTK
stop_words_standard = set(stopwords.words('indonesian'))

# Daftar kata-kata penting yang TIDAK BOLEH dihapus (pengecualian)
list_stopwords_exceptions = [
    'baik','maju','hebat','berhasil','kuat','setuju','mantap','yakin','terus','murah','adil','lancar',
    'mulia','bangga','cemerlang','senang','keren','optimis','menang','sukses','terbukti','sejahtera',
    'bagus','terbaik','solusi','amanah','jujur','peduli','tegas','pintar',
    'tidak','bukan','jangan','belum','gagal','jelek','susah','mahal','rugi','hancur','lemah','salah',
    'keliru','buruk','parah','bingung','mengecewakan','bohong','kacau','nyungsep','menjerit','terpuruk',
    'turun','kurang','goblok','tolol'
]

list_stopwords_include = [
    'jokowi', 'ekonomi', 'prabowosalahkansby', 'pilihorangbaik', 'pilihbajuputih',
    'pilihjelasislamnya', 'jokowimenangtotaldebat'
]
custom_stopwords_exceptions = set(list_stopwords_exceptions)


# --- DAFTAR STOPWORD FINAL ---

# Hapus kata-kata pengecualian dari daftar gabungan. Ini adalah daftar final yang akan kita gunakan.
final_stop_words = (stop_words_standard - custom_stopwords_exceptions) | set(list_stopwords_include)


# --- BUAT SATU FUNGSI PREPROCESSING UTAMA ---

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Hapus karakter non-ASCII (Untuk mengatasi emoji dan mojibake)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    # Hapus URL
    text = re.sub(r'http\S+|www\S+|https\S+|pic.twitter.com\S+|t.co|S+', '', text, flags=re.MULTILINE)
    # Hapus @username dan #hashtag
    text = re.sub(r'\@|#', '', text)
    # Hapus tanda baca, angka, dan karakter spesial
    text = re.sub(r'[^a-zA-Z\s | ? ]', '', text)
    # Normalize menggunakan indo_normalizer
    normalized_tuple = normalizer.normalize_text(text)
    text = normalized_tuple[0] # Extract the normalized text from the tuple


    # Tokenisasi (memecah teks menjadi daftar kata)
    tokens = word_tokenize(text)

    # Menghapus stopwords menggunakan daftar final kita
    tokens = [word for word in tokens if word not in final_stop_words]


    # Menggabungkan kembali token menjadi string
    return ' '.join(tokens)


# --- Eksekusi pada DataFrame ---
df['tweet_bersih']   = df['tweet'].apply(preprocess_text)

print('Hasil setelah preprocessing')
print(df[['tweet','sentimen','tweet_bersih']].head())

print('\n','='*50,'\n')
print('Statistik sentimen setelah preprocessing')
print(df['sentimen'].value_counts())

# Visualisasi Word Cloud

In [ ]:
positif_text_updated = ' '.join(df[df['sentimen'] == 'positif']['tweet_bersih'])
negatif_text_updated = ' '.join(df[df['sentimen'] == 'negatif']['tweet_bersih'])
netral_text_updated = ' '.join(df[df['sentimen'] == 'netral']['tweet_bersih'])

wordcloud_positif_upd = WordCloud(width=800, height=400, background_color='white').generate(positif_text_updated)
wordcloud_negatif_upd = WordCloud(width=800, height=400, background_color='white').generate(negatif_text_updated)
wordcloud_netral_upd = WordCloud(width=800, height=400, background_color='white').generate(netral_text_updated)

plt.figure(figsize=(15,8))

plt.subplot(1, 3, 1)
plt.imshow(wordcloud_positif_upd, interpolation='bilinear')
plt.title('Word Cloud untuk Sentimen Positif')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(wordcloud_negatif_upd, interpolation='bilinear')
plt.title('Word Cloud untuk Sentimen Negatif')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(wordcloud_netral_upd, interpolation='bilinear')
plt.title('Word Cloud untuk Sentimen Netral')
plt.axis('off')

plt.show()

# -- Opsi save file --

In [ ]:
# df[['sentimen', 'tweet_bersih']].to_csv('tweet_bersih.csv', index=False)